# Crypto Data Pipeline: Download and Preparation

This notebook downloads 4-hour OHLCV candle data for hundreds of
cryptocurrencies from **Binance Public Data** and converts the raw CSVs
into partitioned **Parquet** files for efficient Apache Spark processing.

**Steps:**
1. Discover available trading pairs via the Binance API
2. Download monthly 4h candle archives in parallel
3. Process raw CSVs (add headers, convert timestamps)
4. Convert to year/month-partitioned Parquet using Spark
5. Verify the final dataset

## 1. Imports

In [12]:
import requests
import os
import zipfile
import io
import shutil
import glob as globlib
import xml.etree.ElementTree as ET
from datetime import datetime, timezone
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Tuple, Dict, Set

import findspark
try:
    findspark.init()
except Exception:
    pass

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, DoubleType, LongType
)

## 2. Configuration

We target **4-hour candles** from October 2024 through December 2025.
The Oct-Dec 2024 window is warmup data that supports a 90-day lookback
for the backtest starting in January 2025.

In [13]:
# Fallback pair list (used only when live Binance discovery fails)
FALLBACK_TOP_PAIRS = [
    # Top 10 by market cap
    "BTCUSDT", "ETHUSDT", "BNBUSDT", "XRPUSDT", "SOLUSDT",
    "ADAUSDT", "DOGEUSDT", "TRXUSDT", "AVAXUSDT", "LINKUSDT",
    # 11-20
    "DOTUSDT", "MATICUSDT", "UNIUSDT", "LTCUSDT", "ATOMUSDT",
    "ETCUSDT", "XLMUSDT", "FILUSDT", "NEARUSDT", "APTUSDT",
    # 21-30
    "ARBUSDT", "OPUSDT", "VETUSDT", "ALGOUSDT", "FTMUSDT",
    "SANDUSDT", "MANAUSDT", "AXSUSDT", "AAVEUSDT", "EGLDUSDT",
    # 31-40
    "EOSUSDT", "XTZUSDT", "THETAUSDT", "ICPUSDT", "GRTUSDT",
    "FLOWUSDT", "NEOUSDT", "MKRUSDT", "SNXUSDT", "KAVAUSDT",
    # 41-50
    "RNDRUSDT", "INJUSDT", "SUIUSDT", "SEIUSDT", "TIAUSDT",
    "IMXUSDT", "LDOUSDT", "RUNEUSDT", "CFXUSDT", "MINAUSDT",
    # 51-60
    "APEUSDT", "GMXUSDT", "FETUSDT", "AGIXUSDT", "OCEANUSDT",
    "WOOUSDT", "CRVUSDT", "COMPUSDT", "LRCUSDT", "ENJUSDT",
    # 61-70
    "CHZUSDT", "GALAUSDT", "DYDXUSDT", "ZECUSDT", "DASHUSDT",
    "WAVESUSDT", "BATUSDT", "ZILUSDT", "IOSTUSDT", "ONTUSDT",
    # 71-80
    "HOTUSDT", "RVNUSDT", "ZENUSDT", "SCUSDT", "ICXUSDT",
    "ZRXUSDT", "SXPUSDT", "KSMUSDT", "CELRUSDT", "ONEUSDT",
    # 81-90
    "QTUMUSDT", "ANKRUSDT", "SKLUSDT", "COTIUSDT", "BAKEUSDT",
    "IOTAUSDT", "CTSIUSDT", "BANDUSDT", "STMXUSDT", "OGNUSDT",
    # 91-100
    "NKNUSDT", "DENTUSDT", "MTLUSDT", "REEFUSDT", "DGBUSDT",
    "1INCHUSDT", "SUSHIUSDT", "YFIUSDT", "AUDIOUSDT", "CELOUSDT",
]

# Cross-sector seed pairs to enforce diversification
DIVERSIFIED_SEED_PAIRS = {
    "large_cap": [
        "BTCUSDT", "ETHUSDT", "BNBUSDT", "XRPUSDT", "SOLUSDT",
        "ADAUSDT", "TRXUSDT", "DOGEUSDT", "AVAXUSDT", "LINKUSDT",
    ],
    "meme": [
        "DOGEUSDT", "SHIBUSDT", "PEPEUSDT", "FLOKIUSDT", "BONKUSDT",
        "WIFUSDT", "MEMEUSDT", "BOMEUSDT", "1000SATSUSDT", "TURBOUSDT",
    ],
    "stable": ["USDCUSDT", "FDUSDUSDT", "USDPUSDT", "TUSDUSDT", "DAIUSDT"],
    "commodity": ["PAXGUSDT", "XAUTUSDT"],
    "infra_exchange": [
        "BNBUSDT", "ATOMUSDT", "NEARUSDT", "DOTUSDT", "INJUSDT",
        "TIAUSDT", "SEIUSDT", "SUIUSDT", "APTUSDT",
    ],
    "defi_lending_dex": [
        "UNIUSDT", "AAVEUSDT", "MKRUSDT", "LDOUSDT", "CRVUSDT",
        "COMPUSDT", "SNXUSDT", "RUNEUSDT", "DYDXUSDT",
    ],
    "ai_data": ["FETUSDT", "AGIXUSDT", "OCEANUSDT", "TAOUSDT", "RNDRUSDT"],
    "gaming_nft": [
        "IMXUSDT", "AXSUSDT", "SANDUSDT", "MANAUSDT", "GALAUSDT",
        "APEUSDT", "ENJUSDT",
    ],
}

# Time range
START_YEAR, END_YEAR = 2024, 2025
START_MONTH, END_MONTH = 10, 12  # Oct 2024 through Dec 2025
INTERVAL = "4h"

# Binance endpoints
BASE_URL = "https://data.binance.vision/data/spot/monthly/klines"
S3_BUCKET_LIST_URL = "https://s3-ap-northeast-1.amazonaws.com/data.binance.vision"
EXCHANGE_INFO_URL = "https://api.binance.com/api/v3/exchangeInfo"
TICKER_24HR_URL = "https://api.binance.com/api/v3/ticker/24hr"

# Output folders
CSV_FOLDER = "crypto_data_4h"
PARQUET_FOLDER = "crypto_data_parquet"

# Download concurrency
MAX_WORKERS = 10
DEFAULT_QUOTE_ASSET = "USDT"
DEFAULT_TARGET_PAIRS = 300

# Binance CSV columns (after our timestamp conversion)
COLUMNS = [
    "datetime", "open", "high", "low", "close", "volume",
    "quote_volume", "trades", "taker_buy_base",
    "taker_buy_quote", "ignore",
]

## 3. Pair Universe Discovery

We offer four modes for selecting which trading pairs to download:

| Mode | Description |
|------|-------------|
| `top100` | Static fallback list (stable, reproducible) |
| `diversified` | Curated seeds + volume-ranked fill up to target count |
| `all-usdt` | Every active Binance spot pair quoted in USDT |
| `historical-usdt` | All historically listed pairs (includes delisted) |

The **diversified** mode is the default. It seeds the universe from eight
sectors (large-cap, meme, stablecoin, commodity, infra, DeFi, AI, gaming)
and fills the remaining slots by 24-hour quote volume.

In [14]:
def generate_year_months(start_year, start_month, end_year, end_month):
    """Build a list of (year, month) tuples for the download range."""
    periods = []
    for year in range(start_year, end_year + 1):
        m_start = start_month if year == start_year else 1
        m_end = end_month if year == end_year else 12
        for month in range(m_start, m_end + 1):
            periods.append((str(year), f"{month:02d}"))
    return periods


def is_likely_leveraged_token(symbol, quote="USDT"):
    """Heuristic filter for leveraged tokens like BTCUPUSDT, ETHDOWNUSDT."""
    if not symbol.endswith(quote):
        return False
    base = symbol[: -len(quote)]
    tags = {"UP", "DOWN", "BULL", "BEAR"}
    if base in tags:
        return True
    for suffix in tags:
        if base.endswith(suffix) and len(base[: -len(suffix)]) >= 2:
            return True
    return False


def list_s3_common_prefixes(prefix, delimiter="/", timeout=30):
    """Paginate through the Binance public-data S3 bucket listing."""
    prefixes, marker = [], ""
    while True:
        params = {"prefix": prefix, "delimiter": delimiter}
        if marker:
            params["marker"] = marker
        resp = requests.get(S3_BUCKET_LIST_URL, params=params, timeout=timeout)
        resp.raise_for_status()
        root = ET.fromstring(resp.text)
        ns = root.tag.split("}")[0].strip("{") if "}" in root.tag else ""
        tag = (lambda n: f"{{{ns}}}{n}") if ns else (lambda n: n)
        for cp in root.findall(tag("CommonPrefixes")):
            el = cp.find(tag("Prefix"))
            if el is not None and el.text:
                prefixes.append(el.text)
        nm = root.find(tag("NextMarker"))
        marker = nm.text if nm is not None and nm.text else ""
        if not marker:
            break
    return prefixes


def fetch_historical_spot_pairs(quote="USDT", include_leveraged=False):
    """Discover all historically listed USDT spot pairs from the S3 archive."""
    raw = list_s3_common_prefixes("data/spot/monthly/klines/")
    symbols = [p.rstrip("/").split("/")[-1] for p in raw]
    filtered = [s for s in symbols if s.endswith(quote)]
    if not include_leveraged:
        filtered = [s for s in filtered if not is_likely_leveraged_token(s, quote)]
    return sorted(set(filtered))


def fetch_active_spot_pairs(quote="USDT", timeout=20):
    """Fetch currently active spot pairs from the Binance Exchange Info API."""
    resp = requests.get(EXCHANGE_INFO_URL, timeout=timeout)
    resp.raise_for_status()
    data = resp.json()
    return sorted(
        s["symbol"]
        for s in data.get("symbols", [])
        if s.get("status") == "TRADING"
        and s.get("isSpotTradingAllowed")
        and s.get("quoteAsset") == quote
    )


def fetch_volume_ranked_pairs(quote="USDT", timeout=20):
    """Return spot pairs ordered by 24h quote volume (highest first)."""
    resp = requests.get(TICKER_24HR_URL, timeout=timeout)
    resp.raise_for_status()
    ranked = []
    for row in resp.json():
        sym = row.get("symbol", "")
        if sym.endswith(quote):
            try:
                ranked.append((sym, float(row.get("quoteVolume", 0))))
            except (ValueError, TypeError):
                pass
    ranked.sort(key=lambda x: x[1], reverse=True)
    return [sym for sym, _ in ranked]

In [15]:
def _add_unique(pair, selected, seen, active):
    if pair in active and pair not in seen:
        selected.append(pair)
        seen.add(pair)


def build_diversified_universe(active_pairs, target, quote="USDT"):
    """
    Build a diversified pair list:
    1) Seed from each sector category
    2) Fill remaining slots by 24h quote volume
    """
    active_set = set(active_pairs)
    selected, seen = [], set()

    print("\nSeeding diversified universe:")
    for cat, seeds in DIVERSIFIED_SEED_PAIRS.items():
        before = len(selected)
        for p in seeds:
            _add_unique(p, selected, seen, active_set)
        print(f"  {cat}: +{len(selected) - before}")

    # Fill with volume-ranked pairs
    for p in fetch_volume_ranked_pairs(quote):
        if len(selected) >= target:
            break
        _add_unique(p, selected, seen, active_set)

    # Last resort: alphabetical fill
    for p in active_pairs:
        if len(selected) >= target:
            break
        _add_unique(p, selected, seen, active_set)

    return selected[:target]


def resolve_pair_universe(
    mode="diversified",
    target=300,
    quote="USDT",
    include_leveraged_tokens=False,
):
    """
    Resolve the symbol universe for download.

    Modes: top100 | diversified | all-usdt | historical-usdt

    include_leveraged_tokens:
      - Only relevant for historical-usdt mode.
      - Keep False for index research; leveraged tokens are synthetic products and
        can distort volatility/return comparisons.
    """
    mode = mode.lower()

    if mode == "top100":
        pairs = [p for p in FALLBACK_TOP_PAIRS if p.endswith(quote)]
        print(f"Pair mode: top100 (static). {len(pairs)} pairs.")
        return pairs

    if mode == "historical-usdt":
        try:
            pairs = fetch_historical_spot_pairs(
                quote=quote,
                include_leveraged=include_leveraged_tokens,
            )
            print(
                f"Pair mode: historical-usdt. Using {len(pairs)} historical {quote} pairs "
                f"(include_leveraged_tokens={include_leveraged_tokens})."
            )
            return pairs
        except Exception as e:
            print(f"WARNING: historical discovery failed ({e}), falling back.")

    try:
        active = fetch_active_spot_pairs(quote)
        print(f"Discovered {len(active)} active Binance spot {quote} pairs.")

        if mode == "all-usdt":
            return active

        if mode == "diversified":
            cap = min(max(1, target), len(active))
            pairs = build_diversified_universe(active, cap, quote)
            print(f"Pair mode: diversified. {len(pairs)} pairs (target={cap}).")
            return pairs

        raise ValueError(f"Unknown pair mode: {mode}")
    except Exception as e:
        print(f"WARNING: live discovery failed ({e}), using fallback.")
        return [p for p in FALLBACK_TOP_PAIRS if p.endswith(quote)]


## 4. Download Functions

Each monthly archive is a ZIP file hosted on `data.binance.vision`.
We download them in parallel, extract the CSV, and immediately convert
the raw Binance timestamp (milliseconds since epoch) into a
human-readable `YYYY-MM-DD HH:MM:SS` string.

In [16]:
def convert_binance_timestamp(ts):
    """Convert a Binance open_time (int) to a datetime string.
    Handles seconds, milliseconds, and microsecond precision."""
    if ts >= 10**15:
        seconds = ts / 1e6
    elif ts >= 10**12:
        seconds = ts / 1e3
    else:
        seconds = float(ts)
    dt = datetime.utcfromtimestamp(seconds)
    return dt.strftime("%Y-%m-%d %H:%M:%S")


def add_header_to_csv(csv_file, columns):
    """Rewrite a raw Binance CSV: convert timestamps and prepend a header row."""
    try:
        with open(csv_file, "r") as f:
            lines = f.readlines()

        if lines and "datetime" in lines[0]:
            return  # already processed

        tmp = csv_file + ".tmp"
        converted = 0
        with open(tmp, "w", newline="") as f:
            f.write(",".join(columns) + "\n")
            for line in lines:
                if not line.strip():
                    continue
                parts = line.strip().split(",")
                if len(parts) < 12:
                    continue
                if any(ch.isalpha() for ch in parts[0]):
                    continue
                dt_str = convert_binance_timestamp(int(parts[0]))
                # Skip close_time (parts[6]); keep the rest
                row = f"{dt_str},{parts[1]},{parts[2]},{parts[3]},{parts[4]},{parts[5]},"
                row += f"{parts[7]},{parts[8]},{parts[9]},{parts[10]},{parts[11]}\n"
                f.write(row)
                converted += 1

        if converted == 0:
            os.remove(tmp)
            raise ValueError("No rows converted")
        os.replace(tmp, csv_file)

    except Exception as e:
        tmp = csv_file + ".tmp"
        if os.path.exists(tmp):
            try:
                os.remove(tmp)
            except OSError:
                pass
        print(f"  Error processing {csv_file}: {e}")


def download_file(pair, year, month, interval, folder):
    """Download and extract one monthly ZIP from Binance Public Data."""
    file_name = f"{pair}-{interval}-{year}-{month}.zip"
    url = f"{BASE_URL}/{pair}/{interval}/{file_name}"
    try:
        resp = requests.get(url, timeout=30)
        if resp.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
                z.extractall(folder)
            csv_path = os.path.join(folder, file_name.replace(".zip", ".csv"))
            if os.path.exists(csv_path):
                add_header_to_csv(csv_path, COLUMNS)
            return (pair, year, month, True, "OK")
        return (pair, year, month, False, f"HTTP {resp.status_code}")
    except Exception as e:
        return (pair, year, month, False, str(e)[:60])


def download_all_data(pairs, periods, interval, folder, workers=10):
    """Download all pair x month combinations in parallel."""
    os.makedirs(folder, exist_ok=True)
    tasks = [(p, y, m) for p in pairs for y, m in periods]
    total = len(tasks)
    print(f"\nDownloading {total} files ({len(pairs)} pairs x {len(periods)} months) ...")

    ok, fails = 0, []
    with ThreadPoolExecutor(max_workers=workers) as pool:
        futures = {
            pool.submit(download_file, p, y, m, interval, folder): (p, y, m)
            for p, y, m in tasks
        }
        for i, fut in enumerate(as_completed(futures), 1):
            pair, year, month, success, msg = fut.result()
            if success:
                ok += 1
            else:
                fails.append((pair, year, month, msg))
            if i % 50 == 0 or i == total:
                print(f"  Progress: {i}/{total} ({ok} OK)")

    print(f"\nDownload complete. Success: {ok}, Failed: {len(fails)}")
    return {"success": ok, "failed": len(fails)}

## 5. Execute Download Pipeline

Resolve the pair universe and download all monthly archives.

Default is now `historical-usdt` because the backtest needs a realistic 2024-2025 tradable universe (including delisted symbols), not only currently active listings.
Leveraged tokens are excluded by default for cleaner index construction.


In [17]:
# Historical universe is the default for the study:
# - includes pairs that existed in 2024/2025 (not only currently active ones)
# - excludes leveraged tokens to keep assets economically comparable
PAIR_MODE = "historical-usdt"
INCLUDE_LEVERAGED_TOKENS = False

pairs = resolve_pair_universe(
    mode=PAIR_MODE,
    target=DEFAULT_TARGET_PAIRS,
    include_leveraged_tokens=INCLUDE_LEVERAGED_TOKENS,
)
periods = generate_year_months(START_YEAR, START_MONTH, END_YEAR, END_MONTH)
print(f"\nPlan: {len(pairs)} pairs x {len(periods)} months = {len(pairs) * len(periods)} files")

download_all_data(pairs, periods, INTERVAL, CSV_FOLDER, MAX_WORKERS)


Pair mode: historical-usdt. Using 598 historical USDT pairs (include_leveraged_tokens=False).

Plan: 598 pairs x 15 months = 8970 files



/tmp/ipykernel_31617/3480073543.py:10: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(seconds)


  Progress: 50/8970 (33 OK)
  Progress: 100/8970 (73 OK)
  Progress: 150/8970 (112 OK)
  Progress: 200/8970 (161 OK)
  Progress: 250/8970 (209 OK)
  Progress: 300/8970 (248 OK)
  Progress: 350/8970 (270 OK)
  Progress: 400/8970 (305 OK)
  Progress: 450/8970 (342 OK)
  Progress: 500/8970 (380 OK)
  Progress: 550/8970 (410 OK)
  Progress: 600/8970 (437 OK)
  Progress: 650/8970 (472 OK)
  Progress: 700/8970 (522 OK)
  Progress: 750/8970 (572 OK)
  Progress: 800/8970 (608 OK)
  Progress: 850/8970 (651 OK)
  Progress: 900/8970 (687 OK)
  Progress: 950/8970 (701 OK)
  Progress: 1000/8970 (733 OK)
  Progress: 1050/8970 (777 OK)
  Progress: 1100/8970 (804 OK)
  Progress: 1150/8970 (843 OK)
  Progress: 1200/8970 (879 OK)
  Progress: 1250/8970 (894 OK)
  Progress: 1300/8970 (919 OK)
  Progress: 1350/8970 (957 OK)
  Progress: 1400/8970 (982 OK)
  Progress: 1450/8970 (1013 OK)
  Progress: 1500/8970 (1041 OK)
  Progress: 1550/8970 (1088 OK)
  Progress: 1600/8970 (1117 OK)
  Progress: 1650/8970 (114

{'success': 6169, 'failed': 2801}

### Student Interpretation

The downloader found a large historical universe (**598 USDT pairs**) and scheduled **8,970** pair-month files.
This is expected for a historical backtest because we include delisted symbols, so some monthly files naturally do not exist.

In my run, the progress output confirms the job completed end-to-end and produced a broad raw dataset for 2024-10 to 2025-12.


## 6. Convert CSV to Parquet with Spark

We use Spark to read all processed CSVs, extract the trading-pair symbol
from each filename, and write the data as **Snappy-compressed, year/month
partitioned Parquet**. This format enables efficient predicate push-down
when loading specific time ranges later.

In [18]:
spark = (
    SparkSession.builder
    .appName("CryptoDataPrep")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.shuffle.partitions", "10")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")
print("Spark version:", spark.version)

Spark version: 4.1.1


In [19]:
schema = StructType([
    StructField("datetime",        StringType(),  True),
    StructField("open",            DoubleType(),  True),
    StructField("high",            DoubleType(),  True),
    StructField("low",             DoubleType(),  True),
    StructField("close",           DoubleType(),  True),
    StructField("volume",          DoubleType(),  True),
    StructField("quote_volume",    DoubleType(),  True),
    StructField("trades",          LongType(),    True),
    StructField("taker_buy_base",  DoubleType(),  True),
    StructField("taker_buy_quote", DoubleType(),  True),
    StructField("ignore",          DoubleType(),  True),
])

csv_path = os.path.join(CSV_FOLDER, "*.csv")
df = spark.read.csv(csv_path, schema=schema, header=True)

# Extract the trading pair symbol from the filename
# e.g.  .../BTCUSDT-4h-2025-01.csv  ->  BTCUSDT
df = df.withColumn("_fname", F.input_file_name())
df = df.withColumn(
    "symbol",
    F.regexp_extract("_fname", r"([A-Z0-9]+USDT)-\d+h-", 1),
)

# Parse the datetime string into a proper Spark TimestampType
df = df.withColumn("timestamp", F.to_timestamp("datetime", "yyyy-MM-dd HH:mm:ss"))

# Partition columns
df = df.withColumn("year", F.year("timestamp"))
df = df.withColumn("month", F.month("timestamp"))

# Keep only the columns we need
output = df.select(
    "timestamp", "symbol", "open", "high", "low", "close", "volume",
    "year", "month",
).filter(F.col("close").isNotNull())

# Write partitioned Parquet (overwrites any previous output)
if os.path.exists(PARQUET_FOLDER):
    shutil.rmtree(PARQUET_FOLDER)

output.write.partitionBy("year", "month").parquet(PARQUET_FOLDER)
print(f"Parquet written to: {PARQUET_FOLDER}/")

Parquet written to: crypto_data_parquet/


### Student Interpretation

At this stage we convert CSV files into partitioned Parquet (`year`, `month`) for efficient Spark reads.
This is an important big-data design choice: we reduce repeated CSV parsing cost and enable partition pruning during model and backtest windows.


## 7. Verify Parquet Dataset

Read the freshly written Parquet back into Spark and check row counts,
symbol counts, and the date range covered.

In [20]:
parquet_df = spark.read.parquet(PARQUET_FOLDER)
total_rows = parquet_df.count()
n_symbols = parquet_df.select("symbol").distinct().count()

print(f"Total rows:  {total_rows:,}")
print(f"Symbols:     {n_symbols}")

# Show month distribution
parquet_df.groupBy("year", "month").count().orderBy("year", "month").show(20)

Total rows:  1,110,662
Symbols:     501


+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2024|   10|71220|
|2024|   11|69295|
|2024|   12|72108|
|2025|    1|72718|
|2025|    2|66597|
|2025|    3|74209|
|2025|    4|71889|
|2025|    5|73845|
|2025|    6|72556|
|2025|    7|74985|
|2025|    8|75880|
|2025|    9|75109|
|2025|   10|80037|
|2025|   11|78781|
|2025|   12|81433|
+----+-----+-----+



### Student Interpretation

The verification output confirms the prepared dataset size is around **1.11 million rows** across **~500 symbols**.
This is large enough to demonstrate scalable Spark processing while still fitting on our local WSL setup.


In [21]:
spark.stop()
print("Spark session stopped. Data preparation complete.")

Spark session stopped. Data preparation complete.
